### Threading

In [22]:
from threading import Thread
import time


# all threads can access this global variable
database_value = 0

def increase():
    global database_value # needed to modify the global value
    
    # get a local copy (simulate data retrieving)
    local_copy = database_value
        
    # simulate some modifying operation
    local_copy += 1
    time.sleep(0.1)
        
    # write the calculated new value into the global variable
    database_value = local_copy


if __name__ == "__main__":

    print('Start value: ', database_value)

    t1 = Thread(target=increase)
    t2 = Thread(target=increase)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print('End value:', database_value)

    print('end main')

Start value:  0
End value: 1
end main


In [19]:
from threading import Thread, Lock
import time


# all threads can access this global variable
database_value = 0

def increase(lock):
    global database_value # needed to modify the global value
    
    with lock:
        # get a local copy (simulate data retrieving)
        local_copy = database_value
        # simulate some modifying operation
        local_copy += 1
        time.sleep(0.1)
        
        # write the calculated new value into the global variable
        database_value = local_copy


if __name__ == "__main__":
    
    lock = Lock()

    print('Start value: ', database_value)

    t1 = Thread(target=increase, args = (lock,))
    t2 = Thread(target=increase, args = (lock,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print('End value:', database_value)

    print('end main')

Start value:  0
End value: 2
end main


1. **Without lock only 1 thread got executed, with lock the thread executed in synchonized way** <br>
2. `time.sleep(0.1)` **is very important to understand race condition**

In [9]:
from threading import Thread, Lock, current_thread
from queue import Queue

def worker(q, lock):
    while True:
        value = q.get()  # blocks until the item is available

        # do stuff...
        with lock:
            # prevent printing at the same time with this lock
            print(f"in {current_thread().name} got {value}")
        # ...

        # For each get(), a subsequent call to task_done() tells the queue
        # that the processing on this item is complete.
        # If all tasks are done, q.join() can unblock
        q.task_done()


if __name__ == '__main__':
    q = Queue()
    num_threads = 10
    lock = Lock()

    for i in range(num_threads):
        t = Thread(name=f"Thread{i+1}", target=worker, args=(q, lock))
        t.daemon = True  # dies when the main thread dies
        t.start()
    
    # fill the queue with items
    for x in range(20):
        q.put(x)

    q.join()  # Blocks until all items in the queue have been gotten and processed.

    print('main done')

in Thread2 got 0
in Thread2 got 1
in Thread2 got 2
in Thread2 got 3
in Thread2 got 4
in Thread2 got 5
in Thread2 got 6
in Thread2 got 7
in Thread2 got 8
in Thread5 got 9
in Thread4 got 10
in Thread6 got 11
in Thread7 got 12
in Thread9 got 13
in Thread10 got 14
in Thread3 got 15
in Thread8 got 16
in Thread1 got 17
in Thread2 got 18
in Thread5 got 19
main done


### Multiprocessing

In [20]:
from multiprocessing import Process, Value, Array
import time

def add_100(number):
    for _ in range(100):
        time.sleep(0.01)
        number.value += 1

def add_100_array(numbers):
    for _ in range(100):
        time.sleep(0.1)
        for i in range(len(numbers)):
            numbers[i] += 1


if __name__ == "__main__":

    shared_number = Value('i', 0) 
    print('Value at beginning:', shared_number.value)

    shared_array = Array('d', [0.0, 100.0, 200.0])
    print('Array at beginning:', shared_array[:])

    process1 = Process(target=add_100, args=(shared_number,))
    process2 = Process(target=add_100, args=(shared_number,))

    process3 = Process(target=add_100_array, args=(shared_array,))
    process4 = Process(target=add_100_array, args=(shared_array,))

    start_time = time.time()
    process1.start()
    process2.start()
    process3.start()
    process4.start()

    process1.join()
    process2.join()
    process3.join()
    process4.join()
    end_time = time.time()

    print('Value at end:', shared_number.value)
    print('Array at end:', shared_array[:])

    print('end main')
    print('Total multiprocess time', (end_time - start_time))

Value at beginning: 0
Array at beginning: [0.0, 100.0, 200.0]
Value at end: 200
Array at end: [198.0, 299.0, 399.0]
end main
Total multiprocess time 10.038383960723877


**Observations -**
1. The race condition is more probable, for longer processes.
2. The race condition can only be avoided by use of locks or queue.

## How to use `Locks`
Notice that in the above example, the 2 processes should increment the shared value by 1 for 100 times. This results in 200 total operations. But why is the end value not 200?

#### Race condition
A race condition happened here. A race condition occurs when two or more processes or threads can access shared data and they try to change it at the same time. In our example the two processes have to read the shared value, increase it by 1, and write it back into the shared variable. If this happens at the same time, the two processes read the same value, increase it and write it back. Thus, both processes write the same increased value back into the shared object, and the value was not increased by 2. See https://www.python-engineer.com/learn/advancedpython16_threading/ for a detailed explanation of race conditions.

#### Avoid race conditions with `Locks`
A lock (also known as mutex) is a synchronization mechanism for enforcing limits on access to a resource in an environment where there are many processes/threads of execution. A Lock has two states: **locked** and **unlocked**. 
If the state is locked, it does not allow other concurrent processes/threads to enter this code section until the state is unlocked again.

Two functions are important:
- `lock.acquire()` : This will lock the state and block
- `lock.release()` : This will unlock the state again.

Important: You should always release the block again after it was acquired!

In our example the critical code section where the shared variable is read and increased is now locked. This prevents the second process from modyfing the shared object at the same time. Not much has changed in our code. All new changes are commented in the code below.

In [5]:
# import Lock
from multiprocessing import Lock
from multiprocessing import Process, Value, Array
import time

def add_100(number, lock):
    for _ in range(100):
        time.sleep(0.01)
        # lock the state
        lock.acquire()
        
        number.value += 1
        
        # unlock the state
        lock.release()

def add_100_array(numbers, lock):
    for _ in range(100):
        time.sleep(0.01)
        for i in range(len(numbers)):
            lock.acquire()
            numbers[i] += 1
            lock.release()


if __name__ == "__main__":

    # create a lock
    lock = Lock()
    
    shared_number = Value('i', 0) 
    print('Value at beginning:', shared_number.value)

    shared_array = Array('d', [0.0, 100.0, 200.0])
    print('Array at beginning:', shared_array[:])

    # pass the lock to the target function
    process1 = Process(target=add_100, args=(shared_number, lock))
    process2 = Process(target=add_100, args=(shared_number, lock))

    process3 = Process(target=add_100_array, args=(shared_array, lock))
    process4 = Process(target=add_100_array, args=(shared_array, lock))

    start_time = time.time()
    process1.start()
    process2.start()
    process3.start()
    process4.start()

    process1.join()
    process2.join()
    process3.join()
    process4.join()
    end_time = time.time()

    print('Value at end:', shared_number.value)
    print('Array at end:', shared_array[:])

    print('end main')
    print('Total multiprocess time', (end_time - start_time))

Value at beginning: 0
Array at beginning: [0.0, 100.0, 200.0]
Value at end: 200
Array at end: [200.0, 300.0, 400.0]
end main
Total multiprocess time 1.0320227146148682


#### Use the lock as a context manager
After `lock.acquire()` you should never forget to call `lock.release()` to unblock the code. You can also use a lock as a context manager, wich will safely lock and unlock your code. It is recommended to use a lock this way: